<a target="_blank" href="https://colab.research.google.com/github/IngCarlaPezzone/tensorflow-1-public/blob/main/C1/W4/ungraded_labs/C1_W4_Lab_1_image_generator_no_validation_traducido.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Ungraded Lab: Entrenamiento con ImageDataGenerator

En este laboratorio, construirá un modelo de entrenamiento en el conjunto de datos [Horses or Humans](https://www.tensorflow.org/datasets/catalog/horses_or_humans). Éste contiene más de mil imágenes de caballos y humanos con diferentes poses y tamaños de archivos. Utilizarás la clase [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) para preparar este conjunto de datos y alimentar una red neuronal convolucional.

**NOTA IMPORTANTE:** Este cuaderno está diseñado para ejecutarse como Colab. Ejecutarlo en tu máquina local puede dar lugar a que algunos de los bloques de código arrojen errores.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

# Ungraded Lab: Training with ImageDataGenerator

In this lab, you will build a train a model on the [Horses or Humans](https://www.tensorflow.org/datasets/catalog/horses_or_humans) dataset. This contains over a thousand images of horses and humans with varying poses and filesizes. You will use the [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) class to prepare this dataset so it can be fed to a convolutional neural network.

**IMPORTANT NOTE:** This notebook is designed to run as a Colab. Running it on your local machine might result in some of the code blocks throwing errors.

In [ ]:
# Instale este paquete para utilizar la GPU de Colab para la formación
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

Ejecute el siguiente código para descargar el conjunto de datos comprimido `horse-or-human.zip`.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

Run the code below to download the compressed dataset `horse-or-human.zip`.

In [ ]:
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

A continuación, puede descomprimir el archivo con el módulo [zipfile](https://docs.python.org/3/library/zipfile.html).

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

You can then unzip the archive using the [zipfile](https://docs.python.org/3/library/zipfile.html) module.

In [ ]:
import zipfile

# Unzip el dataset
local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')
zip_ref.close()

El contenido del .zip se extrae al directorio base `./caballo-o-humano`, que a su vez contiene subdirectorios `caballo` y `humano`.

En resumen: el conjunto de entrenamiento son los datos que se utilizan para decirle al modelo de la red neuronal que "así es como se ve un caballo" y "así es como se ve un humano".

Hay que prestar atención a este ejemplo: No etiquetamos explícitamente las imágenes como caballos o humanos. En su lugar, se utilizará la API ImageDataGenerator, que está codificada para etiquetar automáticamente las imágenes de acuerdo con los nombres y la estructura del directorio. Así, por ejemplo, tendrá un directorio de "entrenamiento" que contiene un directorio de "caballos" y otro de "humanos". `ImageDataGenerator` etiquetará las imágenes apropiadamente para usted, reduciendo un paso de codificación. 

Ahora puede definir cada uno de estos directorios:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

The contents of the .zip are extracted to the base directory `./horse-or-human`, which in turn each contain `horses` and `humans` subdirectories.

In short: The training set is the data that is used to tell the neural network model that 'this is what a horse looks like' and 'this is what a human looks like'.

One thing to pay attention to in this sample: We do not explicitly label the images as horses or humans. You will use the ImageDataGenerator API instead -- and this is coded to automatically label images according to the directory names and structure. So, for example, you will have a 'training' directory containing a 'horses' directory and a 'humans' one. `ImageDataGenerator` will label the images appropriately for you, reducing a coding step. 

You can now define each of these directories:

In [ ]:
import os

# Directorio con nuestras fotos de caballos de entrenamiento
train_horse_dir = os.path.join('./horse-or-human/horses')

# Directorio con nuestras fotos de humanos de entrenamiento
train_human_dir = os.path.join('./horse-or-human/humans')

Ahora vea cómo son los nombres de los archivos en los directorios de entrenamiento "caballos" y "humanos":

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

Now see what the filenames look like in the `horses` and `humans` training directories:

In [ ]:
train_horse_names = os.listdir(train_horse_dir)
print(train_horse_names[:10])

train_human_names = os.listdir(train_human_dir)
print(train_human_names[:10])

También puede conocer el número total de imágenes de caballos y humanos en los directorios:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

You can also find out the total number of horse and human images in the directories:

In [ ]:
print('total training horse images:', len(os.listdir(train_horse_dir)))
print('total training human images:', len(os.listdir(train_human_dir)))

Ahora echa un vistazo a algunas imágenes para tener una mejor idea de su aspecto. Primero, configura los parámetros de `matplotlib`:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

Now take a look at a few pictures to get a better sense of what they look like. First, configure the `matplotlib` parameters:

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

# Índice para iterar sobre las imágenes
pic_index = 0

Ahora, muestre un lote de 8 imágenes de caballos y 8 de humanos. Puede volver a ejecutar la celda para ver un nuevo lote cada vez:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

Now, display a batch of 8 horse and 8 human pictures. You can rerun the cell to see a fresh batch each time:

In [ ]:
# Configurar matplotlib fig, y el tamaño para adaptarse a 4x4 fotos
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_horse_pix = [os.path.join(train_horse_dir, fname) 
                for fname in train_horse_names[pic_index-8:pic_index]]
next_human_pix = [os.path.join(train_human_dir, fname) 
                for fname in train_human_names[pic_index-8:pic_index]]

for i, img_path in enumerate(next_horse_pix+next_human_pix):
    # Configurar el subplot; los índices del subplot comienzan en 1
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off') # No mostrar los ejes (ni las líneas de la cuadrícula)

    img = mpimg.imread(img_path)
    plt.imshow(img)

plt.show()


## Construir un pequeño modelo desde cero

Ahora puedes definir la arquitectura del modelo que vas a entrenar.

El paso 1 será importar tensorflow.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

## Building a Small Model from Scratch

Now you can define the model architecture that you will train.

Step 1 will be to import tensorflow.

In [ ]:
import tensorflow as tf

A continuación, añade capas convolucionales como en el ejemplo anterior, y aplana el resultado final para alimentar las capas densamente conectadas. Tenga en cuenta que debido a que este es un problema de clasificación de dos clases, es decir, un *problema de clasificación binaria*, terminará su red con una [activación *sigmoide*](https://wikipedia.org/wiki/Sigmoid_function). Esto hace que el valor de salida de su red sea un único escalar entre 0 y 1, codificando la probabilidad de que la imagen actual sea de clase 1 (en oposición a la clase 0).

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

You then add convolutional layers as in the previous example, and flatten the final result to feed into the densely connected layers. Note that because this is a two-class classification problem, i.e. a *binary classification problem*, you will end your network with a [*sigmoid* activation](https://wikipedia.org/wiki/Sigmoid_function). This makes the output value of your network a single scalar between 0 and 1, encoding the probability that the current image is class 1 (as opposed to class 0).

In [ ]:
model = tf.keras.models.Sequential([
    # Obsérvese que la forma de entrada es el tamaño deseado de la imagen 300x300 con 3 bytes de color
    # Esta es la primera convolución
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # La segunda convolución
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # La tercera convolución
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # La cuarta convolución
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # La quinta convolución
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Aplanar los resultados para alimentar una DNN
    tf.keras.layers.Flatten(),
    # Capa oculta de 512 neuronas
    tf.keras.layers.Dense(512, activation='relu'),
    # Sólo 1 neurona de salida. Contendrá un valor de 0-1 donde 
    # 0 para una clase ('caballos') y 1 para la otra ('humanos')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

Puede revisar la arquitectura de la red y las formas de salida con `model.summary()`.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

You can review the network architecture and the output shapes with `model.summary()`.

In [ ]:
model.summary()

La columna "forma de salida" muestra cómo evoluciona el tamaño de su mapa de características en cada capa sucesiva. Como has visto en una lección anterior, las capas de convolución eliminan los píxeles más externos de la imagen, y cada capa de agrupación reduce las dimensiones a la mitad.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

The "output shape" column shows how the size of your feature map evolves in each successive layer. As you saw in an earlier lesson, the convolution layers removes the outermost pixels of the image, and each pooling layer halves the dimensions.

A continuación, configurará las especificaciones para el entrenamiento del modelo. Entrenará el modelo con la pérdida [`binary_crossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy) porque es un problema de clasificación binaria, y la activación final es una sigmoide. (Para un repaso de las métricas de pérdida, vea este [Curso acelerado de aprendizaje automático](https://developers.google.com/machine-learning/crash-course/descending-into-ml/video-lecture).) Usará el optimizador `rmsprop` con una tasa de aprendizaje de `0.001`. Durante el entrenamiento, querrá controlar la precisión de la clasificación.

**NOTA:** En este caso, el uso del [algoritmo de optimización RMSprop](https://wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp) es preferible al [descenso de gradiente estocástico](https://developers.google.com/machine-learning/glossary/#SGD) (SGD), porque RMSprop automatiza el ajuste de la tasa de aprendizaje por nosotros. (Otros optimizadores, como [Adam](https://wikipedia.org/wiki/Stochastic_gradient_descent#Adam) y [Adagrad](https://developers.google.com/machine-learning/glossary/#AdaGrad), también adaptan automáticamente la tasa de aprendizaje durante el entrenamiento, y funcionarían igualmente bien aquí).

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

Next, you'll configure the specifications for model training. You will train the model with the [`binary_crossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy) loss because it's a binary classification problem, and the final activation is a sigmoid. (For a refresher on loss metrics, see this [Machine Learning Crash Course](https://developers.google.com/machine-learning/crash-course/descending-into-ml/video-lecture).) You will use the `rmsprop` optimizer with a learning rate of `0.001`. During training, you will want to monitor classification accuracy.

**NOTE**: In this case, using the [RMSprop optimization algorithm](https://wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp) is preferable to [stochastic gradient descent](https://developers.google.com/machine-learning/glossary/#SGD) (SGD), because RMSprop automates learning-rate tuning for us. (Other optimizers, such as [Adam](https://wikipedia.org/wiki/Stochastic_gradient_descent#Adam) and [Adagrad](https://developers.google.com/machine-learning/glossary/#AdaGrad), also automatically adapt the learning rate during training, and would work equally well here.)

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

### Preprocesamiento de datos

El siguiente paso es configurar los generadores de datos que leerán las imágenes en las carpetas de origen, las convertirán en tensores `float32` y las alimentarán (con sus etiquetas) al modelo. Tendrás un generador para las imágenes de entrenamiento y otro para las de validación. Estos generadores producirán lotes de imágenes de tamaño 300x300 y sus etiquetas (binarias).

Como ya sabrás, los datos que se introducen en las redes neuronales deben ser normalizados de alguna manera para que sean más fáciles de procesar por la red (por ejemplo, no es común introducir píxeles en bruto en una ConvNet). En este caso, preprocesarás las imágenes normalizando los valores de los píxeles para que estén en el rango `[0, 1]` (originalmente todos los valores están en el rango `[0, 255]`).

En Keras, esto se puede hacer a través de la clase `keras.preprocessing.image.ImageDataGenerator` utilizando el parámetro `rescale`. Esta clase `ImageDataGenerator` permite instanciar generadores de lotes de imágenes aumentadas (y sus etiquetas) mediante `.flow(data, labels)` o `.flow_from_directory(directory)`.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

### Data Preprocessing

Next step is to set up the data generators that will read pictures in the source folders, convert them to `float32` tensors, and feed them (with their labels) to the model. You'll have one generator for the training images and one for the validation images. These generators will yield batches of images of size 300x300 and their labels (binary).

As you may already know, data that goes into neural networks should usually be normalized in some way to make it more amenable to processing by the network (i.e. It is uncommon to feed raw pixels into a ConvNet.) In this case, you will preprocess the images by normalizing the pixel values to be in the `[0, 1]` range (originally all values are in the `[0, 255]` range).

In Keras, this can be done via the `keras.preprocessing.image.ImageDataGenerator` class using the `rescale` parameter. This `ImageDataGenerator` class allows you to instantiate generators of augmented image batches (and their labels) via `.flow(data, labels)` or `.flow_from_directory(directory)`.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Todas las imágenes serán reescaladas por 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flujo de imágenes de entrenamiento en lotes de 128 utilizando el generador train_datagen
train_generator = train_datagen.flow_from_directory(
        './horse-or-human/',  # Este es el directorio de origen para las imágenes de entrenamiento
        target_size=(300, 300),  # Todas las imágenes serán redimensionadas a 300x300
        batch_size=128,
        # Dado que utilizamos la pérdida de binary_crossentropy, necesitamos etiquetas binarias
        class_mode='binary')


### Entrenamiento

Puede comenzar el entrenamiento para 15 épocas -- esto puede tardar unos minutos en ejecutarse.

Anote los valores por época.

La `pérdida` y la `precisión` son grandes indicadores del progreso en el entrenamiento. La "pérdida" mide la predicción del modelo actual contra las etiquetas conocidas, calculando el resultado. La "precisión", por otro lado, es la porción de aciertos. 

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

### Training

You can start training for 15 epochs -- this may take a few minutes to run.

Do note the values per epoch.

The `loss` and `accuracy` are great indicators of progress in training. `loss` measures the current model prediction against the known labels, calculating the result. `accuracy`, on the other hand, is the portion of correct guesses. 

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1)

### Predicción del modelo

Ahora eche un vistazo a la ejecución de una predicción utilizando el modelo. Este código le permitirá elegir 1 o más archivos de su sistema de archivos, cargarlos y ejecutarlos a través del modelo, dando una indicación de si el objeto es un caballo o un humano.

**Nota importante:** Debido a algunos problemas de compatibilidad, el siguiente bloque de código dará lugar a un error después de seleccionar las imágenes a cargar si está ejecutando este cuaderno como un `Colab` en el navegador `Safari`. Para todos los demás navegadores, continúe con el siguiente bloque de código e ignore el que le sigue.

Para los usuarios de Safari: por favor, comente u omita el bloque de código de abajo, descomente el siguiente bloque de código y ejecútelo.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

### Model Prediction

Now take a look at actually running a prediction using the model. This code will allow you to choose 1 or more files from your file system, upload them, and run them through the model, giving an indication of whether the object is a horse or a human.

**Important Note:** Due to some compatibility issues, the following code block will result in an error after you select the images(s) to upload if you are running this notebook as a `Colab` on the `Safari` browser. For all other browsers, continue with the next code block and ignore the next one after it.

_For Safari users: please comment out or skip the code block below, uncomment the next code block and run it._

In [ ]:
## BLOQUE DE CÓDIGO PARA NAVEGADORES QUE NO SEAN SAFARI
## USUARIOS DE SAFARI: POR FAVOR, OMITA ESTE BLOQUE Y EJECUTE EL SIGUIENTE EN SU LUGAR

import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
    # predicting images
    path = '/content/' + fn
    img = image.load_img(path, target_size=(300, 300))
    x = image.img_to_array(img)
    x /= 255
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])

    if classes[0]>0.5:
    print(fn + " is a human")
    else:
    print(fn + " is a horse")
 

Los usuarios de `Safari` tendrán que cargar las imágenes manualmente en su espacio de trabajo. Por favor, siga las instrucciones, descomente el bloque de código de abajo y ejecútelo.

Instrucciones para subir imágenes manualmente en un Colab:

1. Seleccione el icono de la `carpeta` en la `barra de menú` de la izquierda.
2. Haga clic en la `carpeta con una flecha apuntando hacia arriba` llamada `..`.
3. Haga clic en la `carpeta` llamada `tmp`.
4. Dentro de la carpeta `tmp`, `crea una nueva carpeta` llamada `imágenes`. Verás la opción `Nueva carpeta` haciendo clic en el botón de menú de los `3 puntos verticales` junto a la carpeta `tmp`.
5. Dentro de la nueva carpeta `imágenes`, sube una(s) imagen(es) de tu elección, preferiblemente de un caballo o de un humano. Arrastra y suelta la(s) imagen(es) encima de la carpeta `images`.
6. Descomente y ejecute el bloque de código que aparece a continuación. 

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

`Safari` users will need to upload the images(s) manually in their workspace. Please follow the instructions, uncomment the code block below and run it.

Instructions on how to upload image(s) manually in a Colab:

1. Select the `folder` icon on the left `menu bar`.
2. Click on the `folder with an arrow pointing upwards` named `..`
3. Click on the `folder` named `tmp`.
4. Inside of the `tmp` folder, `create a new folder` called `images`. You'll see the `New folder` option by clicking the `3 vertical dots` menu button next to the `tmp` folder.
5. Inside of the new `images` folder, upload an image(s) of your choice, preferably of either a horse or a human. Drag and drop the images(s) on top of the `images` folder.
6. Uncomment and run the code block below. 

In [ ]:
# # CODE BLOCK FOR SAFARI USERS

# import numpy as np
# from keras.preprocessing import image
# import os

# images = os.listdir("/tmp/images")

# print(images)

# for i in images:
#  print()
#  # predicting images
#  path = '/tmp/images/' + i
#  img = image.load_img(path, target_size=(300, 300))
#  x = image.img_to_array(img)
#  x /= 255
#  x = np.expand_dims(x, axis=0)

#  images = np.vstack([x])
#  classes = model.predict(images, batch_size=10)
#  print(classes[0])
#  if classes[0]>0.5:
#    print(i + " is a human")
#  else:
#    print(i + " is a horse")

### Visualización de representaciones intermedias

Para tener una idea de qué tipo de características ha aprendido tu CNN, una cosa divertida es visualizar cómo se transforma una entrada a medida que pasa por el modelo.

Puedes elegir una imagen al azar del conjunto de entrenamiento, y luego generar una figura donde cada fila es la salida de una capa, y cada imagen en la fila es un filtro específico en ese mapa de características de salida. Vuelva a ejecutar esta celda para generar representaciones intermedias para una variedad de imágenes de entrenamiento.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

### Visualizing Intermediate Representations

To get a feel for what kind of features your CNN has learned, one fun thing to do is to visualize how an input gets transformed as it goes through the model.

You can pick a random image from the training set, and then generate a figure where each row is the output of a layer, and each image in the row is a specific filter in that output feature map. Rerun this cell to generate intermediate representations for a variety of training images.

In [ ]:
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Definir un nuevo modelo que tomará una imagen como entrada, y emitirá
# representaciones intermedias para todas las capas del modelo anterior después de
# la primera.
successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

# Prepara una imagen de entrada aleatoria del conjunto de entrenamiento.
horse_img_files = [os.path.join(train_horse_dir, f) for f in train_horse_names]
human_img_files = [os.path.join(train_human_dir, f) for f in train_human_names]
img_path = random.choice(horse_img_files + human_img_files)

img = load_img(img_path, target_size=(300, 300))  # this is a PIL image
x = img_to_array(img)  # Numpy array with shape (300, 300, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 300, 300, 3)

# Escala por  1/255
x /= 255

# Pasar la imagen por la red, obteniendo así todas
# representaciones intermedias de esta imagen.
successive_feature_maps = visualization_model.predict(x)

# Estos son los nombres de las capas, por lo que puede tenerlos como parte de la trama
layer_names = [layer.name for layer in model.layers[1:]]

# Visualizar las representaciones
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
    if len(feature_map.shape) == 4:

    # Sólo se hace esto para las capas conv / maxpool, no para las capas totalmente conectadas
    n_features = feature_map.shape[-1]  # número de características en el mapa de características

    # El mapa de características tiene forma (1, size, size, n_features)
    size = feature_map.shape[1]

    # Mosaico de las imágenes en esta matriz
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
        x = feature_map[0, :, :, i]
        x -= x.mean()
        x /= x.std()
        x *= 64
        x += 128
        x = np.clip(x, 0, 255).astype('uint8')

      # Colocar cada filtro en esta gran cuadrícula horizontal
      display_grid[:, i * size : (i + 1) * size] = x

    # Mostrar la rejilla
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

Se puede ver arriba cómo los píxeles resaltados se convierten en representaciones cada vez más abstractas y compactas, especialmente en la red inferior. 

Las representaciones de la parte inferior empiezan a resaltar aquello a lo que la red presta atención, y muestran cada vez menos características que se "activan"; la mayoría se ponen a cero. Esto se llama _representation sparsity_ y es una característica clave del aprendizaje profundo. Estas representaciones llevan cada vez menos información sobre los píxeles originales de la imagen, pero una información cada vez más refinada sobre la clase de la imagen. Se puede pensar en una convnet (o en una red profunda en general) como una tubería de destilación de información en la que cada capa filtra las características más útiles.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

You can see above how the pixels highlighted turn to increasingly abstract and compact representations, especially at the bottom grid. 

The representations downstream start highlighting what the network pays attention to, and they show fewer and fewer features being "activated"; most are set to zero. This is called _representation sparsity_ and is a key feature of deep learning. These representations carry increasingly less information about the original pixels of the image, but increasingly refined information about the class of the image. You can think of a convnet (or a deep network in general) as an information distillation pipeline wherein each layer filters out the most useful features.

## Limpiar

Continuarás con un ejercicio similar en el siguiente laboratorio, pero antes, ejecuta la siguiente celda para terminar el kernel y liberar recursos de memoria:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

## Clean Up

You will continue with a similar exercise in the next lab but before that, run the following cell to terminate the kernel and free memory resources:

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)